In [17]:
import numpy as np
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

In [18]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [19]:
all_words = []
tags = []
patterns = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        patterns.append((w, tag))

# print('### WORDS')
# print(all_words)
# print('### TAGS')
# print(tags)
# print('### PATTERNS')
# print(patterns)

In [20]:
ignore_words = ['?', '!', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

# print(len(patterns), 'patterns')
# print(len(tags), 'tags')
# print(len(all_words), 'unique stemmed words')

In [21]:
x_train = []
y_train = []

for (pattern_sentence, tag) in patterns:
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

In [22]:
epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(x_train[0])
hidden_size = 8
output_size = len(tags)

In [23]:
class ChatData(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [24]:
dataset = ChatData()
trainloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [26]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
for epoch in range(epochs):
    for (words, labels) in trainloader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        optimizer.zero_grad()
        output = model(words)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

print(f'Final Loss: {loss.item():.4f}')

data = {"model_state": model.state_dict(),
        "input_size": input_size,
        "hidden_size": hidden_size,
        "output_size": output_size,
        "all_words": all_words,
        "tags": tags}
file = "trained.pth"
torch.save(data, file)
print(f'Training complete. File saved!')

Epoch [100/1000], Loss: 0.0006
Epoch [200/1000], Loss: 0.0006
Epoch [300/1000], Loss: 0.0002
Epoch [400/1000], Loss: 0.0003
Epoch [500/1000], Loss: 0.0001
Epoch [600/1000], Loss: 0.0001
Epoch [700/1000], Loss: 0.0001
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0001
Epoch [1000/1000], Loss: 0.0000
Final Loss: 0.0000
Training complete. File saved!
